In [1]:
import numpy as np

from utils import OrthPolySyst

### Entanglement entropy

One approach to studying quantum correlations in spin chains is calculating the bipartite entropy of entanglement between a subsystem $A$ of $L$ contiguous particles and the rest of the system, when it is in an energy eigenstate $\rho$:

$$
S = s[\rho_A]
$$

$$
\begin{aligned}
\rho_A &= \mathrm{tr}_{\bar A} \rho, & A &= \{0, ..., L-1\}, & \bar{A} &= \{L, ..., N-1\}
\end{aligned}
$$

for an appropriate entropy functional $s$. Since the system is in an eigenstate of the Hamiltonian, we may suppose a state which contains the $M$ lowest energy excitations, $|M\rangle=\prod_{n=0}^{M-1}\tilde{a}^\dagger_n|0\rangle$ (the corresponding density operator is $\rho=|M\rangle\langle M|$). We can then calculate the $L\times L$ correlation matrix, $C(L, M)\equiv C$ 

$$
\begin{aligned}
C_{nm} &= \langle M|a^\dagger_n a_m|M\rangle, & n,m = 0, ..., L-1
\end{aligned}
$$

The state $|M\rangle$ may always be regarded as the ground state (which is most typical when studying entropy scaling) by adding a constant magnetic field accross all sites, $-h\sum_{n=0}^{N-1} a^\dagger_n a_n$ [3]. This does not change the Hamiltonian's eigenvectors, and therefore $C$ remains the same.

It is easy to check that

$$
\begin{aligned}
\langle M|\tilde{a}^\dagger_n\tilde{a}_m|M\rangle &= \delta_{nm}\chi_M, & \chi_M &=
    \begin{cases}
    1, \quad n\in\{0, ..., M-1\} \\
    0, \quad n\not\in\{0, ..., M-1\}
    \end{cases}
\end{aligned}
$$

And then since $a_n = \sum\limits_{k=0}^{N-1} \Phi_{nk}\tilde{a}_k$

$$
C_{nm} = \sum\limits_{j=0}^{N-1}\sum\limits_{k=0}^{N-1}\Phi_{nj}\Phi_{mk}\delta_{jk}\chi_M=
\sum\limits_{k=0}^{M-1}\Phi_{nk}\Phi_{mk}=
\boxed{
\sum\limits_{k=0}^{M-1}\frac{w_k}{\sqrt{\gamma_n \gamma_m}} P_n(\varepsilon_k)P_m(\varepsilon_k)
}
$$

Since the model's Hamiltonian is Gaussian (quadratic), any correlation function can be expressed in terms of the two-point correlation functions $C_{nm}$ using Wick's theorem [1].

In [2]:
# Calculating correlation matrix
def correlation_matrix(N, J, h, L, M):
    OPS = OrthPolySyst(N, J**2, h)
    P = OPS.polynomials
    E = OPS.roots
    w = OPS.weights
    g = OPS.normalization
    C = np.empty((L, L))
    for n in range(L):
        for m in range(L):
            C[n, m] = sum([w[i] * P(n, E[i]) * P(m, E[i]) / np.sqrt(g[n] * g[m]) for i in range(M)])
    return C.astype(np.float64)

# Example / test
N = 5
J = np.linspace(1, N-1, N-1)
h = 2 * np.ones(N)
L, M = 3, 3
C = correlation_matrix(N, J, h, L, M)
print('Correlation matrix from orthogonal polynomial system: ')
print(np.around(C, 3))

H1 = np.diag(h) + np.diag(J, 1) + np.diag(J, -1)
Phi = np.linalg.eigh(H1)[1][:L, :M]
print('Correlation matrix from diagonalisation: ')
print(np.around(Phi @ Phi.T, 3))

Correlation matrix from orthogonal polynomial system: 
[[ 0.86  -0.261 -0.18 ]
 [-0.261  0.5   -0.386]
 [-0.18  -0.386  0.59 ]]
Correlation matrix from diagonalisation: 
[[ 0.86  -0.261 -0.18 ]
 [-0.261  0.5   -0.386]
 [-0.18  -0.386  0.59 ]]


Refs. [2], [3] then study the spin chain's Rényi entropy $s_\alpha$:

$$
\begin{aligned}
s_\alpha[\rho_A] &= \frac{1}{1-\alpha}\log\mathrm{tr}(\rho^\alpha_A), & \alpha>0
\end{aligned}
$$

$$
s[\rho_A]\equiv\lim\limits_{\alpha\rightarrow1}s_\alpha[\rho_A]=-\mathrm{tr}(\rho_A\log\rho_A)
$$

Let's briefly outline the algorithm explained in ref. [1] for calculating $s[\rho_A]$ from the correlation matrix. Starting with

$$
C_{nm} = \langle a^\dagger_n a_m\rangle = \mathrm{tr}(a^\dagger_n a_m \rho) = \mathrm{tr}_A(a^\dagger_n a_m \rho_A) \equiv \mathrm{tr}(a^\dagger_n a_m \rho_A)
$$

since $C$ is real symmetric, it is orthogonally diagonalisable

$$
\begin{aligned}
G_{nm} = (R^\mathrm{T}CR)_{nm} &= \nu_n \delta_{nm}, &
g_n = \sum\limits_{k=0}^{L-1} R_{kn} a_k
\end{aligned}
$$

$$
G_{nm} = \langle g^\dagger_n g_m\rangle = \mathrm{tr}(g^\dagger_n g_m \rho_A) = \nu_n \delta_{nm}
$$

which implies $\rho_A$ is uncorrelated and can be written as

$$
\rho_A = \rho_{L-1} \otimes ... \otimes \rho_0
$$

In the $\{|0\rangle, |1\rangle\}\equiv\{\mathrm{vacuum}, \mathrm{fermion}\}$ single excitation (rotated) basis, the matrices of our operators are 

$$
\begin{aligned}
g_i &= \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix}, &
g^\dagger_i &= \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix}, &
\rho_i &= \begin{pmatrix} 1-\alpha_i & \beta^*_i \\ \beta_i & \alpha_i \end{pmatrix}
\end{aligned}
$$

We can now calculate the elements of the $\rho_i$ matrices:

$$
\begin{aligned}
\langle M|g_i |M\rangle &= 0, &
\langle M|g_i |M\rangle &= \mathrm{tr}(g_i\rho_i) = \beta_i & \Rightarrow &&
\beta_i &= 0
\end{aligned}
$$

$$
\begin{aligned}
\mathrm{tr}(g^\dagger_i g_i\rho_i) &= \nu_i, &
\mathrm{tr}(g^\dagger_i g_i\rho_i) &= \alpha_i & \Rightarrow &&
\alpha_i = \nu_i
\end{aligned}
$$

And finally

$$
\boxed{
\rho_A = \bigotimes\limits_{i=0}^{L-1} \mathrm{diag}(1-\nu_i, \nu_i)
}
$$

So once we have computed the correlation matrix, we can use it's eigenvalues $\nu_i$ to compute entropy via the formula

$$
\boxed{
S = \sum\limits_{i=0}^{L-1}s^{(2)}(\nu_i)
}
$$

where $s^{(2)}$ is the binary entropy

$$
s_\alpha^{(2)}(x) = \frac{1}{1-\alpha}\log(x^\alpha+(1-x)^\alpha)
$$

$$
s^{(2)}(x) = -x\log x-(1-x)\log(1-x)
$$

Since $C = \Phi_{LM}\Phi_{LM}^\mathrm{T}$ is real, symmetric, positive semi-definite, and so is $I-C$, then $0\leq\nu_i\leq1$, as expected for probablities/populations [2].

In [3]:
@np.vectorize
def bin_entropy(alpha, nu):
    if np.isclose(nu, 0) or np.isclose(nu, 1):
        return 0
    if np.isclose(alpha, 1):
        return - nu * np.log(nu) - (1 - nu) * np.log(1 - nu)
    return np.log(nu ** alpha + (1 - nu) ** alpha) / (1 - alpha)

# Example / test
N = 5
J = np.linspace(1, N-1, N-1)
h = np.ones(N)
L, M = 3, 3
a = 1

C = correlation_matrix(N, J, h, L, M)
nu = np.linalg.eigvalsh(C)
S = np.sum(bin_entropy(a, nu))
print(f'Entanglement entropy: {S}')

Entanglement entropy: 0.45097541485162695


### References

[1] Latorre, J., & Riera, A. (2009). A short review on entanglement in quantum spin systems. *Journal of Physics A: Mathematical and Theoretical, 42(50), 504002*. Retrieved from https://arxiv.org/abs/0906.1499

[2] Finkel, F., & González-López, A. (2020). Inhomogeneous XX spin chains and quasi-exactly solvable models. *Journal of Statistical Mechanics: Theory and Experiment, 2020(9), 093105*. Retrieved from https://arxiv.org/abs/2007.00369

[3] Finkel, F., & González-López, A. (2021). Entanglement entropy of inhomogeneous XX spin chains with algebraic interactions. *Journal of High Energy Physics, 2021(12)*. Retrieved from https://arxiv.org/abs/2107.12200

[4] Jin, B.Q., & Korepin, V. (2004). Quantum Spin Chain, Toeplitz Determinants and the Fisher–Hartwig Conjecture. *Journal of Statistical Physics, 116(1–4), 79–95*. Retrieved from https://arxiv.org/abs/quant-ph/0304108